# Вектор Шепли

*по лекциям кандидата физико-математических наук [Сысоевой Л.Н](https://www.hse.ru/staff/lsysoeva "НИУ ВШЭ") на курсе ["Математические модели политэкономии"](https://www.hse.ru/edu/courses/219871509)* 

In [2]:
from tkinter import *
from itertools import combinations
from itertools import permutations
import pandas as pd
from math import factorial

Сначала рассмотрим общий случай, далее - объединим все в функцию.

Предположим, у нас есть 3 игрока, у каждого из них есть какой-то ресурс (деньги, количество голосов) и они хотят достичь какую-то цель (например, купить вместе какой-то объект).

In [3]:
number_of_participants = 3
resources = [500, 278, 365]
aim = 660

Коалиция – любое подмножество игроков. 

Характеристическая функция коалиции: $V(k)$ – выигрыш, который может получить коалиция $k$.

Посмотрим, на все возможные коалиции (в том числе и пустое множество, в некоторых играх оно важно т.к. есть игры, когда никто не вкладывается, а ресурс есть).

In [4]:
coalitions = []
for i in range(number_of_participants+1):
    coalitions.extend(list(combinations(list(range(1, number_of_participants+1)), i)))
coalitions

[(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]

Игроки могут вкладываться в разных последовательностях. Посмотрим на них:

In [5]:
order = list(permutations(range(1, number_of_participants+1)))
order

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

Сделаем таблицу, в которой будем отмечать, чье вложение является решающим.

In [6]:
table = pd.DataFrame(index=order, columns=list(range(1, number_of_participants+1)))
table

,1,2,3
"(1, 2, 3)",NaN,NaN,NaN
"(1, 3, 2)",NaN,NaN,NaN
"(2, 1, 3)",NaN,NaN,NaN
"(2, 3, 1)",NaN,NaN,NaN
"(3, 1, 2)",NaN,NaN,NaN
"(3, 2, 1)",NaN,NaN,NaN


In [7]:
for i in order:
    x = 0
    for j in i:
        x += resources[j-1] 
        if x >= aim:
            table.loc[i,j] = 1
            break
        else:
            pass

table = table.fillna('-')
table

,1,2,3
"(1, 2, 3)",-,1,-
"(1, 3, 2)",-,-,1
"(2, 1, 3)",1,-,-
"(2, 3, 1)",1,-,-
"(3, 1, 2)",1,-,-
"(3, 2, 1)",1,-,-


В четырех случаях из шести вложение 1 игрока является решающим, в одном из шести – решающее вложение 2 игрока, и в еще одном из шести – игрока 3.

Компонента вектора Шепли для 2 игрока будет расчитана так: 

In [8]:
sum([x for x in list(table.loc[:, 2]) if x == 1]) / len(order) # сумма единиц разделенная на количество комбинаций

0.16666666666666666

Вектор Шепли: 

In [9]:
probs = {}
for i in range(1, number_of_participants+1):
    p = sum([x for x in list(table.loc[:, i]) if x == 1]) / len(order)
    probs[i] = p
probs

{1: 0.6666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666}

In [10]:
for k, v in probs.items():
    print('Компонента вектора Шепли для', k, 'игрока равна', v)

Компонента вектора Шепли для 1 игрока равна 0.6666666666666666
Компонента вектора Шепли для 2 игрока равна 0.16666666666666666
Компонента вектора Шепли для 3 игрока равна 0.16666666666666666


Формула, по которой можно посчитать каждую компоненту: 
    
$$
\varphi_i=\sum_{i\in k, |k| = t}\frac{(t-1)!\cdot(n-t)!}{n!} \cdot [V(k) - V(k\backslash \{i\})]
$$

+ $i\in k$ – все коалиции, в которые входит i-тый игрок
+ $|k| = t$ – количество членов успешных коалиций
+ $[V(k) - V(k\backslash \{i\})]$ – выигрыш коалиции 𝑘 минус выигрыш коалиции 𝑘, когда в ней нет i-го игрока
+ $k\backslash \{i\}$ – k без i

In [11]:
vector = {}
for i in range(1, number_of_participants+1):
    COMP = 0
    t = []
    V_k = []
    
    V_k_ = []

    for c in coalitions:
        if i in c: # считаем количество игроков коалиций, в которые входит игрок
            t.append(len(c))
            V_ = 0
            V = 0
            for j in c:
                
                V+=resources[j-1] # выигрыш коалиции, если в нее входит i-ый игрок
                
                if j != i:
                    V_ += resources[j-1] # выигрыш коалиции, если в нее не входит i-ый игрок
        
            if V >= aim:
                V_k.append(1)
            else:
                V_k.append(0)
                
            if V_ >= aim:
                V_k_.append(1)
            else:
                V_k_.append(0)

    for l in range(len(t)):
        COMP += ((factorial(t[l] - 1) * factorial(number_of_participants - t[l])) / factorial(number_of_participants)) * (V_k[l] - V_k_[l])
        
    vector[i] = COMP 
    
vector

{1: 0.6666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666}

Получили аналогичный вывод. Теперь запишем все в функцию.

## Функция

In [12]:
# Функция для расчета вектора Шепли
def find_vector(number_of_participants, aim, resources):
    resources = list(map(lambda x: int(x.get()), resources))

    coalitions = []
    for i in range(number_of_participants+1):
        coalitions.extend(list(combinations(list(range(1, number_of_participants+1)), i)))

    vector = {}
    for i in range(1, number_of_participants+1):
        COMP = 0
        t = []
        V_k = []
        V_k_ = []

        for c in coalitions:
            if i in c: # считаем количество игроков коалиций, в которые входит игрок
                t.append(len(c))
                V = 0
                V_ = 0
                for j in c:

                    V+=resources[j-1] # выигрыш коалиции, если в нее входит i-ый игрок

                    if j != i:
                        V_ += resources[j-1] # выигрыш коалиции, если в нее не входит i-ый игрок

                if V >= aim:
                    V_k.append(1)
                else:
                    V_k.append(0)

                if V_ >= aim:
                    V_k_.append(1)
                else:
                    V_k_.append(0)

        for l in range(len(t)):
            COMP += ((factorial(t[l] - 1) * factorial(number_of_participants - t[l])) / factorial(number_of_participants)) * (V_k[l] - V_k_[l])

        vector[i] = COMP 
    for i in range(1, number_of_participants+1):
        print('Ресурс', i, 'игрока равен', resources[i-1])
    
    vector_l = []
    for k, v in vector.items():
        vector_l.append(v)
        print('Компонента вектора Шепли для', k, 'игрока равна', v)
    print('Вектор Шепли:', tuple(vector_l))
    print('==================================')
    print('==================================')

# Функция для ввода данных
def find_vector_menu():
    number_of_participants = int(input('Введите число участников: '))
    aim = int(input('Цель участников (число): '))

    window = Tk()
    window.title("Ресурс каждого игрока")

    l = Label(window, text = 'Player')
    l.grid(row = 0, column = 0)

    l = Label(window, text = 'Resources')
    l.grid(row = 0, column = 1)

    for i in range(number_of_participants):
        l = Label(window, text = str(i+1))
        l.grid(row = i+1, column = 0)

    resources = []    

    for i in range(number_of_participants):
        l = Label(window, text = ' ')
        l.grid(row = i+1, column = 1)

        resources.append(Entry(window, textvariable = IntVar(), width=5))
        resources[-1].grid(row = i+1, column = 1)


    b = Button(window, text = 'Done', width=10, activebackground = '#73f5ee', command=lambda: find_vector(number_of_participants, aim, resources))
    b.grid(row = number_of_participants+1, column = 1)

    window.mainloop()



In [13]:
find_vector_menu() # введем кол-во игроков = 3, цель = 660, ресурсы сначала те, которые рассматривали выше, потом поменяем

Введите число участников: 3
Цель участников (число): 660
Ресурс 1 игрока равен 500
Ресурс 2 игрока равен 278
Ресурс 3 игрока равен 365
Компонента вектора Шепли для 1 игрока равна 0.6666666666666666
Компонента вектора Шепли для 2 игрока равна 0.16666666666666666
Компонента вектора Шепли для 3 игрока равна 0.16666666666666666
Вектор Шепли: (0.6666666666666666, 0.16666666666666666, 0.16666666666666666)
Ресурс 1 игрока равен 300
Ресурс 2 игрока равен 450
Ресурс 3 игрока равен 80
Компонента вектора Шепли для 1 игрока равна 0.5
Компонента вектора Шепли для 2 игрока равна 0.5
Компонента вектора Шепли для 3 игрока равна 0.0
Вектор Шепли: (0.5, 0.5, 0.0)
Ресурс 1 игрока равен 50
Ресурс 2 игрока равен 650
Ресурс 3 игрока равен 800
Компонента вектора Шепли для 1 игрока равна 0.16666666666666666
Компонента вектора Шепли для 2 игрока равна 0.16666666666666666
Компонента вектора Шепли для 3 игрока равна 0.6666666666666666
Вектор Шепли: (0.16666666666666666, 0.16666666666666666, 0.6666666666666666)
Р

In [14]:
find_vector_menu() # поменяем число участников и цель

Введите число участников: 6
Цель участников (число): 1500
Ресурс 1 игрока равен 500
Ресурс 2 игрока равен 350
Ресурс 3 игрока равен 1000
Ресурс 4 игрока равен 800
Ресурс 5 игрока равен 700
Ресурс 6 игрока равен 650
Компонента вектора Шепли для 1 игрока равна 0.13333333333333333
Компонента вектора Шепли для 2 игрока равна 0.08333333333333333
Компонента вектора Шепли для 3 игрока равна 0.2833333333333333
Компонента вектора Шепли для 4 игрока равна 0.18333333333333332
Компонента вектора Шепли для 5 игрока равна 0.18333333333333332
Компонента вектора Шепли для 6 игрока равна 0.13333333333333333
Вектор Шепли: (0.13333333333333333, 0.08333333333333333, 0.2833333333333333, 0.18333333333333332, 0.18333333333333332, 0.13333333333333333)
Ресурс 1 игрока равен 50
Ресурс 2 игрока равен 350
Ресурс 3 игрока равен 100
Ресурс 4 игрока равен 80
Ресурс 5 игрока равен 700
Ресурс 6 игрока равен 650
Компонента вектора Шепли для 1 игрока равна 0.016666666666666666
Компонента вектора Шепли для 2 игрока равна